## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [2]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'
sheet_value = 'TOP VALOR'
sheet_packs = 'TOP PACKS'
sheet_services = 'TOP SERVICIOS'

# Range to read (change this for each project)
start_row = 8
end_row = 150
usecols = 'A:AC'

print("Variables defined correctly!")

Variables defined correctly!


## Verify Columns in 'Active' Sheet.

Verify the number of columns in the "Active" sheet to ensure the range is within bounds. We will use the range in all three sheets

In [ ]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

## Extract Data from all six Sheets

Extract data from all three Sheets within the specified range and convert it directly to a DataFrame.

In [3]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)
value_df = load_sheet_as_dataframe(excel_file_path, 'TOP VALOR', start_row, end_row, usecols)
packs_df = load_sheet_as_dataframe(excel_file_path, 'TOP PACKS', start_row, end_row, usecols)
services_df = load_sheet_as_dataframe(excel_file_path, 'TOP SERVICIOS', start_row, end_row, usecols)

# Display the DataFrames
print("Active DataFrame:")
display(active_df.head(99))

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

print("value DataFrame:")
display(value_df.head())

print("Packs DataFrame:")
display(packs_df.head())

print("Services DataFrame:")
display(services_df.head())


Data from 'ACTIVOS' sheet loaded successfully.
Data from 'CANCELADOS' sheet loaded successfully.
Data from 'PTE FIRMA' sheet loaded successfully.
Data from 'TOP VALOR' sheet loaded successfully.
Data from 'TOP PACKS' sheet loaded successfully.
Data from 'TOP SERVICIOS' sheet loaded successfully.
Active DataFrame:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,albaaraujo@originaltelecom.es,9.0,9.0,NaN,NaN,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,NaN,NaN,34.0,13.0,227
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,4.0,4.0,11
3,albertocanto@originaltelecom.es,8.0,7.0,NaN,NaN,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,NaN,NaN,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,NaN,NaN,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,NaN,NaN,20.0,25.0,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,vanesa.alvarezmartin@cgi.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2
87,victor.justamante@finetwork.com,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6
88,yicel.patricia@originaltelecom.es,NaN,NaN,16.0,7.0,9.0,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,NaN,15.0,19.0,16.0,NaN,165
89,yzabelly.gomes@originaltelecom.es,NaN,NaN,14.0,15.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,21.0,17.0,NaN,NaN,114


Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,165,157,79,64,175,172,185,170,143,...,182,168,183,161,69,49,14,57,30,3322
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,albaaraujo@originaltelecom.es,2.0,NaN,NaN,NaN,6.0,6.0,6.0,4.0,10.0,...,3.0,7.0,8.0,9.0,NaN,NaN,NaN,NaN,3.0,90
2,alberto.valle@finetwork.com,NaN,3.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
3,albertocanto@originaltelecom.es,1.0,7.0,NaN,NaN,NaN,6.0,6.0,5.0,6.0,...,4.0,16.0,2.0,4.0,2.0,NaN,NaN,2.0,NaN,83
4,albertosanchez@originaltelecom.es,5.0,1.0,NaN,NaN,5.0,4.0,10.0,3.0,6.0,...,11.0,4.0,2.0,8.0,1.0,NaN,NaN,3.0,2.0,100


Pending Signature DataFrame:


...

,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,215.0,187.0,103.0,64.0,261.0,240.0,222.0,185.0,200.0,...,315.0,254.0,275.0,277.0,255.0,119.0,86.0,435.0,416.0,5439
1,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,albaaraujo@originaltelecom.es,3.0,4.0,NaN,NaN,1.0,15.0,2.0,4.0,6.0,...,7.0,12.0,6.0,11.0,11.0,NaN,NaN,22.0,13.0,149
3,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,4.0,4.0,10
4,albertocanto@originaltelecom.es,4.0,7.0,NaN,NaN,11.0,4.0,7.0,7.0,7.0,...,10.0,4.0,8.0,7.0,12.0,NaN,NaN,12.0,20.0,158


Packs DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,244.0,223.0,123.0,93.0,274.0,275.0,280.0,230.0,261.0,...,422.0,335.0,368.0,325.0,340.0,152.0,121.0,542.0,486.0,6666
1,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,albaaraujo@originaltelecom.es,6.0,5.0,NaN,NaN,5.0,18.0,3.0,3.0,11.0,...,3.0,10.0,8.0,11.0,18.0,NaN,NaN,33.0,11.0,178
3,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,3.0,4.0,9
4,albertocanto@originaltelecom.es,7.0,6.0,NaN,NaN,6.0,3.0,14.0,10.0,11.0,...,17.0,10.0,10.0,8.0,10.0,NaN,NaN,15.0,18.0,197


Services DataFrame:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
0,alba.salgado@originaltelecom.es,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,albaaraujo@originaltelecom.es,9.0,9.0,NaN,NaN,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,NaN,NaN,34.0,13.0,227
2,alberto.valle@finetwork.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,4.0,4.0,11
3,albertocanto@originaltelecom.es,8.0,7.0,NaN,NaN,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,NaN,NaN,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,NaN,NaN,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,NaN,NaN,20.0,25.0,298


## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [4]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)
value_df = replace_nan_with_zero(value_df)
packs_df = replace_nan_with_zero(packs_df)
services_df = replace_nan_with_zero(services_df)



# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head(88))

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

print("Value DataFrame after replacing NaN:")
display(value_df.head())

print("Packs DataFrame after replacing NaN:")
display(packs_df.head())

print("Services DataFrame after replacing NaN:")
display(services_df.head())

Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Active DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,11
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,sara.elkhelyfy@originaltelecom.es,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,18.0,19.0,20.0,21.0,0.0,0.0,25.0,40.0,174
84,sergio.vazquez@originaltelecom.es,8.0,5.0,0.0,0.0,19.0,11.0,16.0,6.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135
85,tamara.garcia@finetwork.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
86,vanesa.alvarezmartin@cgi.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2


Canceled DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,165,157,79,64,175,172,185,170,143,...,182,168,183,161,69,49,14,57,30,3322
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,6.0,6.0,6.0,4.0,10.0,...,3.0,7.0,8.0,9.0,0.0,0.0,0.0,0.0,3.0,90
2,alberto.valle@finetwork.com,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
3,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,6.0,6.0,5.0,6.0,...,4.0,16.0,2.0,4.0,2.0,0.0,0.0,2.0,0.0,83
4,albertosanchez@originaltelecom.es,5.0,1.0,0.0,0.0,5.0,4.0,10.0,3.0,6.0,...,11.0,4.0,2.0,8.0,1.0,0.0,0.0,3.0,2.0,100


Pending Signature DataFrame after replacing NaN:


...

,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,215.0,187.0,103.0,64.0,261.0,240.0,222.0,185.0,200.0,...,315.0,254.0,275.0,277.0,255.0,119.0,86.0,435.0,416.0,5439
1,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,7.0,12.0,6.0,11.0,11.0,0.0,0.0,22.0,13.0,149
3,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,10
4,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,10.0,4.0,8.0,7.0,12.0,0.0,0.0,12.0,20.0,158


Packs DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,244.0,223.0,123.0,93.0,274.0,275.0,280.0,230.0,261.0,...,422.0,335.0,368.0,325.0,340.0,152.0,121.0,542.0,486.0,6666
1,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,10.0,8.0,11.0,18.0,0.0,0.0,33.0,11.0,178
3,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,4.0,9
4,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,14.0,10.0,11.0,...,17.0,10.0,10.0,8.0,10.0,0.0,0.0,15.0,18.0,197


Services DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
0,alba.salgado@originaltelecom.es,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227
2,alberto.valle@finetwork.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,11
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298


## Load the list of agents

In [5]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originalt

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [6]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)
value_df = ensure_all_agents(value_df, agents_list)
packs_df = ensure_all_agents(packs_df, agents_list)
services_df = ensure_all_agents(services_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head(88))

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head())

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head())

print("Value DataFrame after ensuring all agents:")
display(value_df.head())

print("Packs DataFrame after ensuring all agents:")
display(packs_df.head())

print("Services DataFrame after ensuring all agents:")
display(services_df.head())

Added 2 missing agents and removed 48 agents not in the list.
Added 1 missing agents and removed 44 agents not in the list.
Added 8 missing agents and removed 3 agents not in the list.
Added 2 missing agents and removed 35 agents not in the list.
Added 2 missing agents and removed 42 agents not in the list.
Added 2 missing agents and removed 65 agents not in the list.
Active DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298
7,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,9.0,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198
9,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,19.0,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158
10,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15.0,18.0,0.0,0.0,100
11,beatriz.gomez@originaltelecom.es,6.0,9.0,0.0,0.0,5.0,6.0,6.0,8.0,14.0,...,8.0,9.0,22.0,6.0,12.0,0.0,0.0,18.0,18.0,197
13,carolinafuentes@originaltelecom.es,8.0,8.0,0.0,0.0,11.0,15.0,10.0,15.0,14.0,...,19.0,9.0,21.0,8.0,15.0,0.0,0.0,23.0,9.0,251
15,cesar.arnaldo@originaltelecom.es,0.0,0.0,16.0,7.0,5.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,0.0,0.0,19.0,19.0,17.0,0.0,150
20,david.molero@originaltelecom.es,6.0,5.0,0.0,0.0,17.0,15.0,25.0,16.0,24.0,...,23.0,15.0,22.0,11.0,32.0,0.0,0.0,21.0,23.0,320


Canceled DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,165,157,79,64,175,172,185,170,143,...,182,168,183,161,69,49,14,57,30,3322
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,6.0,6.0,6.0,4.0,10.0,...,3.0,7.0,8.0,9.0,0.0,0.0,0.0,0.0,3.0,90
3,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,6.0,6.0,5.0,6.0,...,4.0,16.0,2.0,4.0,2.0,0.0,0.0,2.0,0.0,83
4,albertosanchez@originaltelecom.es,5.0,1.0,0.0,0.0,5.0,4.0,10.0,3.0,6.0,...,11.0,4.0,2.0,8.0,1.0,0.0,0.0,3.0,2.0,100
5,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,...,7.0,7.0,7.0,0.0,0.0,7.0,3.0,2.0,0.0,102
6,antonio.reina@originaltelecom.es,2.0,12.0,0.0,0.0,9.0,10.0,4.0,1.0,10.0,...,2.0,6.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,96


Pending Signature DataFrame after ensuring all agents:


...

,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
2,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,7.0,12.0,6.0,11.0,11.0,0.0,0.0,22.0,13.0,149
4,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,10.0,4.0,8.0,7.0,12.0,0.0,0.0,12.0,20.0,158
5,albertosanchez@originaltelecom.es,10.0,6.0,0.0,0.0,6.0,15.0,16.0,12.0,6.0,...,18.0,6.0,17.0,7.0,5.0,0.0,0.0,19.0,21.0,204
6,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,2.0,5.0,12.0,0.0,0.0,11.0,6.0,7.0,11.0,127
7,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,6.0,...,12.0,8.0,4.0,5.0,4.0,0.0,0.0,0.0,0.0,102


Packs DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
2,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,10.0,8.0,11.0,18.0,0.0,0.0,33.0,11.0,178
4,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,14.0,10.0,11.0,...,17.0,10.0,10.0,8.0,10.0,0.0,0.0,15.0,18.0,197
5,albertosanchez@originaltelecom.es,11.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,17.0,11.0,20.0,5.0,9.0,0.0,0.0,16.0,20.0,231
6,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,7.0,9.0,24.0,0.0,0.0,7.0,5.0,12.0,13.0,153
7,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,6.0,...,12.0,9.0,7.0,10.0,4.0,0.0,0.0,0.0,0.0,119


Services DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,9.0,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,19.0,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [7]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'maricarmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)
value_df = add_email_values(value_df, email_values)
packs_df = add_email_values(packs_df, email_values)
services_df = add_email_values(services_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head())

print("Canceled DataFrame with email values:")
display(canceled_df.head())

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head())

print("Value DataFrame with email values:")
display(value_df.head())

print("Packs DataFrame with email values:")
display(packs_df.head())

print("Services DataFrame with email values:")
display(services_df.head())

Active DataFrame with email values:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,404,451,413,421,204,149,664,573,8304,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241,2
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298,3
7,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198,4
9,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158,5


Canceled DataFrame with email values:


,Inbound Telec.Orig.Sevilla,165,157,79,64,175,172,185,170,143,...,168,183,161,69,49,14,57,30,3322,email_value
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,6.0,6.0,6.0,4.0,10.0,...,7.0,8.0,9.0,0.0,0.0,0.0,0.0,3.0,90,1
3,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,6.0,6.0,5.0,6.0,...,16.0,2.0,4.0,2.0,0.0,0.0,2.0,0.0,83,2
4,albertosanchez@originaltelecom.es,5.0,1.0,0.0,0.0,5.0,4.0,10.0,3.0,6.0,...,4.0,2.0,8.0,1.0,0.0,0.0,3.0,2.0,100,3
5,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,...,7.0,7.0,0.0,0.0,7.0,3.0,2.0,0.0,102,4
6,antonio.reina@originaltelecom.es,2.0,12.0,0.0,0.0,9.0,10.0,4.0,1.0,10.0,...,6.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,96,5


Pending Signature DataFrame with email values:


...

,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general,email_value
2,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,12.0,6.0,11.0,11.0,0.0,0.0,22.0,13.0,149,1
4,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,4.0,8.0,7.0,12.0,0.0,0.0,12.0,20.0,158,2
5,albertosanchez@originaltelecom.es,10.0,6.0,0.0,0.0,6.0,15.0,16.0,12.0,6.0,...,6.0,17.0,7.0,5.0,0.0,0.0,19.0,21.0,204,3
6,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,5.0,12.0,0.0,0.0,11.0,6.0,7.0,11.0,127,4
7,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,6.0,...,8.0,4.0,5.0,4.0,0.0,0.0,0.0,0.0,102,5


Packs DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general,email_value
2,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,10.0,8.0,11.0,18.0,0.0,0.0,33.0,11.0,178,1
4,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,14.0,10.0,11.0,...,10.0,10.0,8.0,10.0,0.0,0.0,15.0,18.0,197,2
5,albertosanchez@originaltelecom.es,11.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,11.0,20.0,5.0,9.0,0.0,0.0,16.0,20.0,231,3
6,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,9.0,24.0,0.0,0.0,7.0,5.0,12.0,13.0,153,4
7,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,6.0,...,9.0,7.0,10.0,4.0,0.0,0.0,0.0,0.0,119,5


Services DataFrame with email values:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,404,451,413,421,204,149,664,573,8304,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241,2
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298,3
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198,4
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158,5


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [8]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)
value_df = sort_by_email_value(value_df)
packs_df = sort_by_email_value(packs_df)
services_df = sort_by_email_value(services_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head()) 

print("Sorted Canceled DataFrame:")
display(canceled_df.head())

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head())

print("Sorted Canceled DataFrame:")
display(value_df.head())

print("Sorted Pending Signature DataFrame:")
display(packs_df.head())

print("Sorted Pending Signature DataFrame:")
display(services_df.head())

Sorted Active DataFrame:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,404,451,413,421,204,149,664,573,8304,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241,2
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298,3
7,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198,4
9,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158,5


Sorted Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,165,157,79,64,175,172,185,170,143,...,168,183,161,69,49,14,57,30,3322,email_value
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,6.0,6.0,6.0,4.0,10.0,...,7.0,8.0,9.0,0.0,0.0,0.0,0.0,3.0,90,1
3,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,6.0,6.0,5.0,6.0,...,16.0,2.0,4.0,2.0,0.0,0.0,2.0,0.0,83,2
4,albertosanchez@originaltelecom.es,5.0,1.0,0.0,0.0,5.0,4.0,10.0,3.0,6.0,...,4.0,2.0,8.0,1.0,0.0,0.0,3.0,2.0,100,3
5,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,...,7.0,7.0,0.0,0.0,7.0,3.0,2.0,0.0,102,4
6,antonio.reina@originaltelecom.es,2.0,12.0,0.0,0.0,9.0,10.0,4.0,1.0,10.0,...,6.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,96,5


Sorted Pending Signature DataFrame:


...

,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general,email_value
2,albaaraujo@originaltelecom.es,3.0,4.0,0.0,0.0,1.0,15.0,2.0,4.0,6.0,...,12.0,6.0,11.0,11.0,0.0,0.0,22.0,13.0,149,1
4,albertocanto@originaltelecom.es,4.0,7.0,0.0,0.0,11.0,4.0,7.0,7.0,7.0,...,4.0,8.0,7.0,12.0,0.0,0.0,12.0,20.0,158,2
5,albertosanchez@originaltelecom.es,10.0,6.0,0.0,0.0,6.0,15.0,16.0,12.0,6.0,...,6.0,17.0,7.0,5.0,0.0,0.0,19.0,21.0,204,3
6,anasanchez@originaltelecom.es,0.0,0.0,12.0,5.0,7.0,7.0,9.0,0.0,0.0,...,5.0,12.0,0.0,0.0,11.0,6.0,7.0,11.0,127,4
7,antonio.reina@originaltelecom.es,9.0,2.0,0.0,0.0,5.0,1.0,9.0,13.0,6.0,...,8.0,4.0,5.0,4.0,0.0,0.0,0.0,0.0,102,5


Sorted Pending Signature DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general,email_value
2,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,10.0,8.0,11.0,18.0,0.0,0.0,33.0,11.0,178,1
4,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,14.0,10.0,11.0,...,10.0,10.0,8.0,10.0,0.0,0.0,15.0,18.0,197,2
5,albertosanchez@originaltelecom.es,11.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,11.0,20.0,5.0,9.0,0.0,0.0,16.0,20.0,231,3
6,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,9.0,24.0,0.0,0.0,7.0,5.0,12.0,13.0,153,4
7,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,6.0,...,9.0,7.0,10.0,4.0,0.0,0.0,0.0,0.0,119,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,404,451,413,421,204,149,664,573,8304,email_value
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227,1
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241,2
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298,3
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198,4
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158,5


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [9]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)
value_df = remove_email_value_column(value_df)
packs_df = remove_email_value_column(packs_df)
services_df = remove_email_value_column(services_df)


# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head())

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head())

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head())

print("Value DataFrame after removing 'email_value' column:")
display(value_df.head())

print("Packs DataFrame after removing 'email_value' column:")
display(packs_df.head())

print("Services DataFrame after removing 'email_value' column:")
display(services_df.head())

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

Active DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298
7,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,9.0,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198
9,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,19.0,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158


Canceled DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,165,157,79,64,175,172,185,170,143,...,182,168,183,161,69,49,14,57,30,3322
1,albaaraujo@originaltelecom.es,2.0,0.0,0.0,0.0,6.0,6.0,6.0,4.0,10.0,...,3.0,7.0,8.0,9.0,0.0,0.0,0.0,0.0,3.0,90
3,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,6.0,6.0,5.0,6.0,...,4.0,16.0,2.0,4.0,2.0,0.0,0.0,2.0,0.0,83
4,albertosanchez@originaltelecom.es,5.0,1.0,0.0,0.0,5.0,4.0,10.0,3.0,6.0,...,11.0,4.0,2.0,8.0,1.0,0.0,0.0,3.0,2.0,100
5,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,...,7.0,7.0,7.0,0.0,0.0,7.0,3.0,2.0,0.0,102
6,antonio.reina@originaltelecom.es,2.0,12.0,0.0,0.0,9.0,10.0,4.0,1.0,10.0,...,2.0,6.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,96


Pending Signature DataFrame after removing 'email_value' column:


...

Packs DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,...,2024-08-19 00:00:00,2024-08-20 00:00:00,2024-08-21 00:00:00,2024-08-22 00:00:00,2024-08-23 00:00:00,2024-08-24 00:00:00,2024-08-25 00:00:00,2024-08-26 00:00:00,2024-08-27 00:00:00,Total general
2,albaaraujo@originaltelecom.es,6.0,5.0,0.0,0.0,5.0,18.0,3.0,3.0,11.0,...,3.0,10.0,8.0,11.0,18.0,0.0,0.0,33.0,11.0,178
4,albertocanto@originaltelecom.es,7.0,6.0,0.0,0.0,6.0,3.0,14.0,10.0,11.0,...,17.0,10.0,10.0,8.0,10.0,0.0,0.0,15.0,18.0,197
5,albertosanchez@originaltelecom.es,11.0,8.0,0.0,0.0,5.0,7.0,25.0,21.0,9.0,...,17.0,11.0,20.0,5.0,9.0,0.0,0.0,16.0,20.0,231
6,anasanchez@originaltelecom.es,0.0,0.0,11.0,8.0,15.0,9.0,11.0,0.0,0.0,...,7.0,9.0,24.0,0.0,0.0,7.0,5.0,12.0,13.0,153
7,antonio.reina@originaltelecom.es,3.0,3.0,0.0,0.0,4.0,6.0,13.0,11.0,6.0,...,12.0,9.0,7.0,10.0,4.0,0.0,0.0,0.0,0.0,119


Services DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,322,269,167,106,349,348,338,280,307,...,510,404,451,413,421,204,149,664,573,8304
1,albaaraujo@originaltelecom.es,9.0,9.0,0.0,0.0,5.0,21.0,4.0,4.0,11.0,...,7.0,13.0,11.0,17.0,23.0,0.0,0.0,34.0,13.0,227
3,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,7.0,15.0,10.0,11.0,...,20.0,11.0,12.0,10.0,14.0,0.0,0.0,17.0,22.0,241
4,albertosanchez@originaltelecom.es,14.0,10.0,0.0,0.0,10.0,15.0,29.0,23.0,10.0,...,21.0,14.0,22.0,8.0,13.0,0.0,0.0,20.0,25.0,298
8,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,...,9.0,12.0,27.0,0.0,0.0,12.0,7.0,13.0,15.0,198
10,antonio.reina@originaltelecom.es,9.0,5.0,0.0,0.0,5.0,6.0,13.0,17.0,8.0,...,19.0,11.0,8.0,12.0,6.0,0.0,0.0,0.0,0.0,158


Values from the second column of active_df:
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroy

## Process Data from all DataFrames

Sum all numeric values in each row, divide the result by 2, and store the final values.

In [10]:
# Calculate sum of numeric values in each row, divided by 2
active_sums = active_df.iloc[:, 1:].sum(axis=1) / 2
canceled_sums = canceled_df.iloc[:, 1:].sum(axis=1) / 2
pending_signature_sums = pending_signature_df.iloc[:, 1:].sum(axis=1) / 2
value_sums = value_df.iloc[:, 1:].sum(axis=1) / 2
packs_sums = packs_df.iloc[:, 1:].sum(axis=1) / 2
services_sums = services_df.iloc[:, 1:].sum(axis=1) / 2

print("Calculated sums for 'Active' DataFrame:")
print(active_sums.head(66))# Calculate sum of numeric values in each row, divided by 2 for 'Pending Signature' DataFrame
print("Calculated sums for 'Canceled' DataFrame:")
print(canceled_sums.head())
print("Calculated sums for 'Pending Signature' DataFrame:")
print(pending_signature_sums.head())
print("Calculated sums for 'Value' DataFrame:")
print(value_sums.head())
print("Calculated sums for 'Packs' DataFrame:")
print(packs_sums.head())
print("Calculated sums for 'Services' DataFrame:")
print(services_df.head())

Calculated sums for 'Active' DataFrame:
1     227.0
3     241.0
4     298.0
7     198.0
9     158.0
10    100.0
11    197.0
61    168.0
13    251.0
15    150.0
20    320.0
24    256.0
27    174.0
37    270.0
39    314.0
40    223.0
42    151.0
44     99.0
50    225.0
49    120.0
52    210.0
23    128.0
56    186.0
91      0.0
92      0.0
60    228.0
59    236.0
58    161.0
63    112.0
64    183.0
67    267.0
68    204.0
57    120.0
71    187.0
72    160.0
73    159.0
74    174.0
77     85.0
78    169.0
83    174.0
84    135.0
88    165.0
89    114.0
dtype: float64
Calculated sums for 'Canceled' DataFrame:
1     90.0
3     83.0
4    100.0
5    102.0
6     96.0
dtype: float64
Calculated sums for 'Pending Signature' DataFrame:
0     3.0
1     8.0
2    11.0
3     3.0
4     2.0
dtype: float64
Calculated sums for 'Value' DataFrame:
2    149.0
4    158.0
5    204.0
6    127.0
7    102.0
dtype: float64
Calculated sums for 'Packs' DataFrame:
2    178.0
4    197.0
5    231.0
6    153.0
7    119.

## Update finalFile Excel Sheet

Update the "finalFile" Excel sheet with the calculated values from all five DataFrame

### Update finalFile Excel sheet with Active Data

Update the "finalFile" Excel sheet with the calculated values from the Active DataFrame in column J.

In [11]:
from openpyxl import load_workbook

# Path to the final file
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'  # Change this to the correct sheet name

def update_final_file(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column F (6th column)
    start_row = 3
    column = 10  # Column J
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}'.")

# Update the final file with the calculated sums
update_final_file(final_file_path, sheet_name, active_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'.


### Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column K.

In [12]:
def update_final_file_canceled(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column N (14th column)
    start_row = 3
    column = 11  # Column K
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Update the final file with the calculated sums for Canceled
update_final_file_canceled(final_file_path, sheet_name, canceled_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Canceled data.


### Update finalFile Excel Sheet with Pending Signature Data

Update the "finalFile" Excel sheet with the calculated values from the Pending Signature DataFrame in column L.

In [13]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 12  # Column L
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Pending Signature data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, pending_signature_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Pending Signature data.


### Update finalFile Excel Sheet with Values Data

Update the "finalFile" Excel sheet with the calculated values from the Values DataFrame in column M.

In [14]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 13  # Column M
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Values data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, value_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Values data.


### Update finalFile Excel Sheet with Packs Data

Update the "finalFile" Excel sheet with the calculated values from the Packs DataFrame in column N.

In [15]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 14  # Column N
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Packs data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, packs_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Packs data.


### Update finalFile Excel Sheet with Services Data

Update the "finalFile" Excel sheet with the calculated values from the Services DataFrame in column O.

In [16]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 15  # Column O
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Services data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, services_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Services data.
